In [ ]:
# Introducing the convolutional network

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

# Convert 3D tensor to 1D tensor
model.add(layers.Flatten())

# Analyze the data using the dense layer
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [ ]:
# Let's analyze the mnist data
from keras.datasets import mnist
from keras.utils.all_utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32')/255
test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 55s 58ms/step - loss: 0.1706 - accuracy: 0.9463
Epoch 2/5
938/938 [==============================] - 49s 52ms/step - loss: 0.0465 - accuracy: 0.9859
Epoch 3/5
938/938 [==============================] - 48s 51ms/step - loss: 0.0326 - accuracy: 0.9905
Epoch 4/5
938/938 [==============================] - 48s 51ms/step - loss: 0.0240 - accuracy: 0.9929
Epoch 5/5
938/938 [==============================] - 47s 50ms/step - loss: 0.0184 - accuracy: 0.9943


In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 3s 9ms/step - loss: 0.0276 - accuracy: 0.9900


0.9900000095367432

In [ ]:
# dogs and cats 데이터 불러오기
# C:\Users\rlsot\OneDrive - unist.ac.kr\바탕 화면\칭화대 수업\220117 여름방학 공부
import os

base_dir = '/Users'
os.path.join(base_dir,'cats')

'/Users/cats'

In [1]:
from keras.applications import vgg16

conv_base = vgg16.VGG16(weights = 'imagenet',
                  include_top = False,
                  input_shape = (150,150,3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               2097408   
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Freezing

conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

# Augment the data
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
'''
test_datagen = ImageDataGenerator(rescale = 1./255)
# training 할 땐 왜곡을 주어 데이터량을 늘린다.
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
)
model.compile(loss = 'binary_crossentropy',
              optimizer = optimizers.RMSprop(lr=2e-5)),
              metrics=['acc'])
# 파일위치로부터 사진 불러오는 기능

history = model.fit_generator(
    train_generator,
    steps_per_epochs = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50
)
'''

In [ ]:
# Fine-tuning

# Freezing all layers up to a specific one
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False


In [ ]:
conv_base.summary()
# block5_conv1,2,3 은 trainable 하게 바꼈고,
# 나머지는 untrainable 하게 설정됨.

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
'''
# Fine-tuning the model
model.compile(loss = 'binary_crossentropy',
              optimizer = optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epochs = 100,
    epochs = 100,
    validation_data = validation_generator,
    validation_steps = 50
)
'''

"\n# Fine-tuning the model\nmodel.compile(loss = 'binary_crossentropy',\n              optimizer = optimizers.RMSprop(lr=1e-5),\n              metrics=['acc'])\n\nhistory = model.fit_generator(\n    train_generator,\n    steps_per_epochs = 100,\n    epochs = 100,\n    validation_data = validation_generator,\n    validation_steps = 50\n)\n"

In [ ]:
'''
# Instantiating a model from an input tensor and a list of output tensors



from keras import models

layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(img_tensor)

first_layer_activation = activations[0]
import matplotlib.pyplot as plt
# the results from the fourth channel
plt.matshow(first_layer_activation[0,:,:,4], cmap = 'viridis')
# the results from the seventh channel
plt.matshow(first_layer_activation[0,:,:,7], cmat = 'viridis')
'''

"\n# Instantiating a model from an input tensor and a list of output tensors\n\n\n\nfrom keras import models\n\nlayer_outputs = [layer.output for layer in model.layers[:8]]\nactivation_model = models.Model(inputs=model.input, outputs=layer_outputs)\nactivations = activation_model.predict(img_tensor)\n\nfirst_layer_activation = activations[0]\nimport matplotlib.pyplot as plt\n# the results from the fourth channel\nplt.matshow(first_layer_activation[0,:,:,4], cmap = 'viridis')\n# the results from the seventh channel\nplt.matshow(first_layer_activation[0,:,:,7], cmat = 'viridis')\n"

In [ ]:
'''
# Visualizing every channel in every intermediate activation
import matplotlib.pyplot as plt
import numpy as np
layer_names = []
for layer in model.layers[:8]:
  layer_names.append(layer.name)
images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations):
  n_features = layer_activation.shape[-1]
  size = layer_activation.shape[1]
  n_cols = n_features//images_per_row
  display_grid = np.zeros((size*n_cols, images_per_row*size))

  for col in range(n_cols):
    for row in range(images_per_row):
      channel_image = layer_activation[0, :, :, col*images_per_row + row]
      channel_image -= channel_image.mean()
      channel_image /= channel_image.std()
      channel_image *= 64
      channel_image += 128
      channel_image = np.clip(channel_image, 0, 255).astype('uint8')
      display_grid[col*size : (col+1)*size, row*size : (row+1)*size] = channel_image

  scale = 1./size
  plt.figure(figsize=(scale*display_grid.shape[1], scale*display_grid.shape[0]))
  plt.title(layer_name)
  plt.grid(False)
  plt.imshow(display_grid, aspect='auto', cmap='viridis')
'''


"\n# Visualizing every channel in every intermediate activation\nimport matplotlib.pyplot as plt\nimport numpy as np\nlayer_names = []\nfor layer in model.layers[:8]:\n  layer_names.append(layer.name)\nimages_per_row = 16\n\nfor layer_name, layer_activation in zip(layer_names, activations):\n  n_features = layer_activation.shape[-1]\n  size = layer_activation.shape[1]\n  n_cols = n_features//images_per_row\n  display_grid = np.zeros((size*n_cols, images_per_row*size))\n\n  for col in range(n_cols):\n    for row in range(images_per_row):\n      channel_image = layer_activation[0, :, :, col*images_per_row + row]\n      channel_image -= channel_image.mean()\n      channel_image /= channel_image.std()\n      channel_image *= 64\n      channel_image += 128\n      channel_image = np.clip(channel_image, 0, 255).astype('uint8')\n      display_grid[col*size : (col+1)*size, row*size : (row+1)*size] = channel_image\n\n  scale = 1./size\n  plt.figure(figsize=(scale*display_grid.shape[1], scale*dis

In [ ]:
'''
# Defining the loss tensor for filter visualization
from keras.applications.vgg16 import VGG16
from keras import backend as K

model = VGG16(weights='imagenet',
              include_top = False)
layer_name = 'block3_conv1'
filter_index = 0

layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:,:,:,filter_index])

grads = K.gradients(loss, model.input)[0]
grads /= (K.sqrt(K.mean(K.square(grads)))+1e-5) # L2 normalization
# iterate - loss 와 gradient 리턴함
iterate = K.function([model.input],[loss,grads])
import numpy as np
loss_value, grads_value = iterate([np.zeros((1,150,150,3))])

''' 

"\n# Defining the loss tensor for filter visualization\nfrom keras.applications.vgg16 import VGG16\nfrom keras import backend as K\n\nmodel = VGG16(weights='imagenet',\n              include_top = False)\nlayer_name = 'block3_conv1'\nfilter_index = 0\n\nlayer_output = model.get_layer(layer_name).output\nloss = K.mean(layer_output[:,:,:,filter_index])\n\ngrads = K.gradients(loss, model.input)[0]\ngrads /= (K.sqrt(K.mean(K.square(grads)))+1e-5)\n"

RuntimeError: ignored